In [56]:
# Apikey = bbbc81fe7ca147fd926e408403866f08

import pandas as pd
import requests
import json
import networkx as nx
from time import sleep

In [57]:
#building the request(s) for the api

Api = "https://api.jikan.moe/v4/recommendations/manga"
#apikey = "&apikey=bbbc81fe7ca147fd926e408403866f08"

#getting news about apple starting from today in the us,
#sorted by popularity

resp = requests.get(
    Api 
     
    )

res = resp.json()
res['data']

[{'mal_id': '123475-147901',
  'entry': [{'mal_id': 123475,
    'url': 'https://myanimelist.net/manga/123475/The_Remarried_Empress',
    'images': {'jpg': {'image_url': 'https://cdn.myanimelist.net/images/manga/1/235295.jpg',
      'small_image_url': 'https://cdn.myanimelist.net/images/manga/1/235295t.jpg',
      'large_image_url': 'https://cdn.myanimelist.net/images/manga/1/235295l.jpg'},
     'webp': {'image_url': 'https://cdn.myanimelist.net/images/manga/1/235295.webp',
      'small_image_url': 'https://cdn.myanimelist.net/images/manga/1/235295t.webp',
      'large_image_url': 'https://cdn.myanimelist.net/images/manga/1/235295l.webp'}},
    'title': 'The Remarried Empress'},
   {'mal_id': 147901,
    'url': 'https://myanimelist.net/manga/147901/I_Dont_Love_You_Anymore',
    'images': {'jpg': {'image_url': 'https://cdn.myanimelist.net/images/manga/2/262500.jpg',
      'small_image_url': 'https://cdn.myanimelist.net/images/manga/2/262500t.jpg',
      'large_image_url': 'https://cdn.my

In [58]:
graph = nx.Graph() # Build the graph

In [59]:
#creating edges based on the reccommendation

for each in res['data']:
    
    left_title =   each['entry'][0]['title']
    left_id =      each['entry'][0]['mal_id']
    right_title =  each['entry'][1]['title']
    right_id =     each['entry'][1]['mal_id']

    
    #add both sides as a node in the graph
    if ((left_id,left_title)) not in graph.nodes:
        graph.add_node((left_id,left_title))
    if ((right_id,right_title)) not in graph.nodes:
        graph.add_node((right_id,right_title))

    #create and edge between them if not there already
    if [(left_id,left_title),(right_id,right_title)] not in graph.edges:
        graph.add_edge((left_id,left_title),(right_id,right_title),weight =1)


In [60]:
print("Nodes:", len(graph.nodes))
print(graph.degree())
degsor = sorted(graph.degree(), key=lambda x: x[1], reverse=True)
print(degsor)


Nodes: 174
[((123475, 'The Remarried Empress'), 1), ((147901, "I Don't Love You Anymore"), 1), ((149333, 'Maou to Yuusha no Tatakai no Ura de'), 2), ((151053, 'Barbarian Quest'), 1), ((107795, 'Genjitsu Shugi Yuusha no Oukoku Saikenki'), 1), ((126287, 'Sousou no Frieren'), 1), ((156601, 'Hoshi Tabi Shounen'), 1), ((656, 'Vagabond'), 3), ((3285, 'Holyland'), 1), ((2062, 'Kara no Kyoukai: The Garden of Sinners'), 1), ((26027, 'Mahoutsukai no Yoru'), 1), ((89357, 'Youkoso Jitsuryoku Shijou Shugi no Kyoushitsu e'), 1), ((123992, 'Youkoso Jitsuryoku Shijou Shugi no Kyoushitsu e: 2-nensei-hen'), 1), ((9982, 'Usogui'), 1), ((61579, 'Tobaku Datenroku Kaiji: One Poker-hen'), 1), ((44, 'Gintama'), 1), ((6812, 'Kyou kara Ore wa!!'), 1), ((1649, 'Liar Game'), 1), ((11327, 'One Outs'), 1), ((67223, 'Paradise View'), 1), ((107539, 'Life: Senjou no Bokura'), 1), ((135545, 'Ao no Hako'), 1), ((144267, 'Kaoru Hana wa Rin to Saku'), 1), ((147333, 'Perfect Marriage Revenge'), 1), ((151165, 'Marry My Husb

In [61]:
manga_dict = dict()
for each in graph.nodes:
    print(each)
    resp = requests.get(
    f"https://api.jikan.moe/v4/manga/{each[0]}/full"
     
    )

    res = resp.json()
    print(res)
    id =res['data']['mal_id']
    manga_dict[res['data']['mal_id']] = res['data']
    sleep(1)

(123475, 'The Remarried Empress')
{'data': {'mal_id': 123475, 'url': 'https://myanimelist.net/manga/123475/The_Remarried_Empress', 'images': {'jpg': {'image_url': 'https://cdn.myanimelist.net/images/manga/1/235295.jpg', 'small_image_url': 'https://cdn.myanimelist.net/images/manga/1/235295t.jpg', 'large_image_url': 'https://cdn.myanimelist.net/images/manga/1/235295l.jpg'}, 'webp': {'image_url': 'https://cdn.myanimelist.net/images/manga/1/235295.webp', 'small_image_url': 'https://cdn.myanimelist.net/images/manga/1/235295t.webp', 'large_image_url': 'https://cdn.myanimelist.net/images/manga/1/235295l.webp'}}, 'approved': True, 'titles': [{'type': 'Default', 'title': 'The Remarried Empress'}, {'type': 'Synonym', 'title': 'Jaehon Hwanghu'}, {'type': 'Japanese', 'title': '재혼 황후'}, {'type': 'English', 'title': 'The Remarried Empress'}], 'title': 'The Remarried Empress', 'title_english': 'The Remarried Empress', 'title_japanese': '재혼 황후', 'title_synonyms': ['Jaehon Hwanghu'], 'type': 'Manhwa', 

In [62]:
genre_map = dict()
theme_map = dict()


for mal_id in manga_dict:
    #print(mal_id)
    gen_list = []
    if len(manga_dict[mal_id]['genres']) > 0:
        for each in manga_dict[mal_id]['genres']:
            gen_list.append(each['name'])
        #print (gen_list)
    genre_map[mal_id] = gen_list


for mal_id in manga_dict:
    #print(mal_id)
    theme_list = []
    if len(manga_dict[mal_id]['themes']) > 0:
        for each in manga_dict[mal_id]['themes']:
            theme_list.append(each['name'])
        # print (gen_list)

    theme_map[mal_id] = theme_list
    

#print(genre_map)
print(theme_map)
    #print(manga_dict[mal_id]['genres'])

{123475: [], 147901: [], 149333: ['Isekai', 'Military', 'Reincarnation', 'Video Game'], 151053: ['Gore'], 107795: ['Isekai', 'Military'], 126287: [], 156601: [], 656: ['Historical', 'Samurai'], 3285: ['Delinquents', 'Martial Arts', 'Psychological'], 2062: ['Mythology', 'Psychological', 'School'], 26027: [], 89357: ['Psychological', 'School'], 123992: ['Psychological', 'School'], 9982: ['High Stakes Game', 'Psychological', 'Strategy Game'], 61579: ['Adult Cast', 'High Stakes Game', 'Psychological', 'Strategy Game'], 44: ['Gag Humor', 'Historical', 'Parody', 'Samurai'], 6812: ['Delinquents', 'School'], 1649: ['High Stakes Game', 'Psychological', 'Strategy Game'], 11327: ['Adult Cast', 'Psychological', 'Team Sports'], 67223: [], 107539: [], 135545: ['School', 'Team Sports'], 144267: ['School'], 147333: ['Time Travel'], 151165: ['Psychological', 'Time Travel'], 1: ['Adult Cast', 'Psychological'], 936: ['Psychological'], 17791: [], 136474: [], 25124: [], 101762: [], 1133: ['Gore'], 100035: 

In [63]:
manga_id_map= dict()
for mal_id in manga_dict:
        manga_id_map[mal_id] = ({
                "manga": manga_dict[mal_id]['title'],
                "popularity": manga_dict[mal_id]['popularity'],
                "score": manga_dict[mal_id]['score'],
                "rank" : manga_dict[mal_id]['rank'],
                "favorites" : manga_dict[mal_id]['favorites'],
                "genres": genre_map[mal_id],
                "status" : manga_dict[mal_id]['status'],
                "synopsis" : manga_dict[mal_id]['synopsis'],
                
            })
        
    #print(mal_id)
    # print(f"\t{manga_dict[mal_id]['title']}")
    # print(f"\t{manga_dict[mal_id]['popularity']}")
    # print(f"\t{manga_dict[mal_id]['score']}")
    # print(f"\t{manga_dict[mal_id]['scored']}")
    # print(f"\t{manga_dict[mal_id]['rank']}")
    # print(f"\t{manga_dict[mal_id]['favorites']}")
    # print(f"\t{manga_dict[mal_id]['synopsis']}")
    # print(f"\t{manga_dict[mal_id]['status']}")


In [64]:
print(manga_id_map)
df = pd.DataFrame.from_dict(manga_id_map, orient = 'index' )
df.head(25)

{123475: {'manga': 'The Remarried Empress', 'popularity': 853, 'score': 8.04, 'rank': 676, 'favorites': 617, 'genres': ['Drama', 'Fantasy', 'Romance'], 'status': 'Publishing', 'synopsis': 'Empress of the Eastern Nation Navier Ellie Trovi stands at the pinnacle of noblewomen with dignity and grace. Although she and her husband, Emperor Sovieshu, are not as intimate as other couples, their trust in each other has been built over years of friendship. But then, wanting something more, he brings in a concubine—a former slave named Rashta—and that is when everything begins to fall apart. Months later, the fateful day arrives; Sovieshu declares his intentions to divorce Navier and give her throne to Rashta. However, against all expectations, Navier is not backed against a wall, and she declares her intention to marry another man—Heinrey—whose noble appearance shocks all in the room. Just who is Heinrey, and how did Navier come to this new marriage arrangement? The Eastern Nation stands at a p

,manga,popularity,score,rank,favorites,genres,status,synopsis
123475,The Remarried Empress,853,8.04,676.0,617,"[Drama, Fantasy, Romance]",Publishing,Empress of the Eastern Nation Navier Ellie Tro...
147901,I Don't Love You Anymore,11591,6.58,14456.0,2,"[Fantasy, Romance]",Publishing,What happens after you fall out of love? Nevae...
149333,Maou to Yuusha no Tatakai no Ura de,17944,7.24,5438.0,4,"[Action, Fantasy]",Publishing,A modern Japanese man reincarnated into a game...
151053,Barbarian Quest,6682,7.47,3090.0,22,"[Action, Adventure, Fantasy]",Publishing,"The greatest warrior of the Boulder Axe Tribe,..."
107795,Genjitsu Shugi Yuusha no Oukoku Saikenki,897,7.45,3319.0,149,"[Action, Fantasy]",Publishing,"""O, Hero!"" With that cliched line, Kazuya Soum..."
126287,Sousou no Frieren,134,8.68,57.0,5110,"[Adventure, Award Winning, Drama, Fantasy]",Publishing,"The demon king has been defeated, and the vict..."
156601,Hoshi Tabi Shounen,47543,NaN,40104.0,1,"[Fantasy, Sci-Fi, Slice of Life]",Publishing,A planet of slumber where most of the inhabita...
656,Vagabond,15,9.24,3.0,40198,"[Action, Adventure, Award Winning]",On Hiatus,"In 16th-century Japan, Shinmen Takezou is a wi..."
3285,Holyland,170,8.45,173.0,3783,[Action],Finished,"A tormented soul, Yuu Kamishiro has no place i..."
2062,Kara no Kyoukai: The Garden of Sinners,1114,8.53,118.0,529,"[Fantasy, Horror, Mystery, Supernatural]",Finished,"A teenage girl, who possesses the ""Mystic Eyes..."


In [54]:
df.to_csv('manga_output')